In [ ]:
#importing 
import numpy as np
import pandas as pd

from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator as data_augment
from tensorflow.keras import layers as layers

#data augmetation 
data_generate_training = data_augment (rescale=1./255, 
                              shear_range = 0.2,
                              zoom_range = 0.2,
                              fill_mode = "nearest",
                              horizontal_flip = True,
                              width_shift_range = 0.2,
                              height_shift_range = 0.2,
                              validation_split = 0.15)

data_generate_test = data_augment(rescale = 1./255)

#data preprocessing and augmentation
traind = data_generate_training.flow_from_directory("Data\Training",
                                          target_size = (96, 96),
                                          seed = 123,
                                          batch_size = 32,
                                          subset = "training")

testd = data_generate_training.flow_from_directory("Data\Validation",
                                          target_size = (96, 96),
                                          seed = 123,
                                          batch_size = 32,
                                          subset = "validation")
#Found 39959 images belonging to 2 classes.
#Found 1747 images belonging to 2 classes.


In [ ]:
#Building Model
CNNmodel = keras.Sequential([
    layers.Conv2D(32, (3, 3), input_shape=(96, 96, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.2),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.2),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.2),
    
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.2),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.2), 
    layers.Dense(256, activation = 'relu'),
    layers.Dense(128, activation = 'relu'),
    layers.Dense(2, activation = 'sigmoid')
])

CNNmodel.compile(optimizer='adam',
              loss="binary_crossentropy",
              metrics=['accuracy'])



In [ ]:

history = CNNmodel.fit(traind, epochs = 10, validation_data = testd) 

In [ ]:
score = CNNmodel.evaluate(testd)

In [ ]:
print(score[1])

In [ ]:
CNNmodel.save('D:\study\FPTU\projAIP\model')

In [ ]:
from matplotlib import pyplot as plt
plot_folder = "plot"
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.1, 1])
plt.legend(loc='lower right')

In [ ]:
plot_folder = "plot"
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Val Loss')
plt.legend(loc='lower right')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

Y_pred = CNNmodel.predict(testd)
y_pred = np.argmax(Y_pred, axis=1)
print(classification_report(testd.classes, y_pred))
print(testd.classes)
print(y_pred)
print('Confusion Matrix')
print(confusion_matrix(testd.classes, y_pred))